In [11]:
import pygame
import random
import time

# 1. 초기화 -> 변수선언
pygame.init()

# 2. 게임화면 설정 -> 크기 고정
size = [400, 200]
screen = pygame.display.set_mode(size)
title = "Using PyGame"

pygame.display.set_caption(title)

# 3. 게임 화면 내부 설정 -> 변수
clock = pygame.time.Clock()

class Object:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.distance = 0
    
    def add_img(self, address):
        if address[-3:] == 'png':
            self.image = pygame.image.load(address).convert_alpha()
        else:
            self.image = pygame.image.load(address)
    
    def change_size(self, width, heigth):
        self.image = pygame.transform.scale(self.image, (width, heigth))
        self.width, self.height = self.image.get_size()
    
    def show(self):
        screen.blit(self.image, (self.x, self.y))

# a.x - b.width <= b.x <= a.x + a.width
# a.y - b.heigth <= b.y <= a.y + a.height
def crash(a, b):
    if (a.x - b.width <= b.x) and (b.x <= a.x + a.width):
        if (a.y - b.height <= b.y) and (b.y <= a.y + a.height):
            return True
        else:
            return False
    else:
        return False

player = Object()
player.add_img("C:/Users/Administrator/Pictures/pygame/player.png")
player.change_size(50, 80)
player.x = round(size[0] / 2) - round(player.width / 2)
player.y = size[1] - player.height - 50
player.distance = 10

left_move = False
right_move = False
space_move = False

black = (0, 0, 0)
rocket_list = []
enemy_list = []
k = 0

# 4. 메인 이벤트
system_exit = 0

while(system_exit == 0):
    #  4 - 1) FPS(Frame Per Second) 설정
    clock.tick(60)
    
    #  4 - 2) 입력(키보드, 마우스) 감지
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            system_exit = 1
            
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                left_move = True
                print(pygame.K_LEFT)
            if event.key == pygame.K_RIGHT:
                right_move = True
                print(pygame.K_RIGHT)
            if event.key == pygame.K_SPACE:
                space_move = True
                print(pygame.K_SPACE)
                k = 0
        
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_move = False
                print(pygame.K_LEFT)
            if event.key == pygame.K_RIGHT:
                right_move = False
                print(pygame.K_RIGHT)
            if event.key == pygame.K_SPACE:
                space_move = False
                print(pygame.K_SPACE) 

    #  4 - 3) 변화(입력 / 시간에 따른 변화)
    if left_move == True:
        player.x -= player.distance
        if player.x <= 0:
            player.x = 0
    elif right_move == True:
        player.x += player.distance
        if player.x >= size[0] - player.width:
            player.x = size[0] - player.width
    
    if space_move == True and k % 6 == 0:
        rocket = Object()
        rocket.add_img("C:/Users/Administrator/Pictures/pygame/rocket.png")
        rocket.change_size(10, 20)
        rocket.x = round(player.x + player.width / 2 - rocket.width / 2)
        rocket.y = player.y - rocket.height - 10
        rocket.distance = 17
        rocket_list.append(rocket)
    
    k += 1
#     print(k)
    
    delete_list = []
    for i in range(len(rocket_list)):
        r = rocket_list[i]
        r.y -= r.distance
        if r.y <= -r.height:
            delete_list.append(r)
    try:
        delete_list.reverse()
        for d in delete_list:
            del rocket_list[d]
    except:
        pass
    
    if random.random() > 0.98:
        enemy = Object()
        enemy.add_img("C:/Users/Administrator/Pictures/pygame/enemy.png")
        enemy.change_size(40, 40)
        enemy.x = random.randrange(0, size[0] - enemy.width - round(player.width / 2))
        enemy.y = 15
        enemy.distance = 5
        enemy_list.append(enemy)
        
        for i in range(len(enemy_list)):
            e = enemy_list[i]
            e.y += e.distance
            if e.y <= -e.height:
                delete_list.append(e)
        try:
            delete_list.reverse()
            for d in delete_list:
                del enemy_list[d]
        except:
            pass
    
    delete_rocket_list = []
    delete_enemy_list = []
    for i in range(len(rocket_list)):
        for j in range(len(enemy_list)):
            r = rocket_list[i]
            e = enemy_list[j]
            if crash(r, e) == True:
                delete_rocket_list.append(i)
                delete_enemy_list.append(j)
    
    delete_rocket_list = list(set(delete_rocket_list))
    delete_enemy_list = list(set(delete_enemy_list))
    
    try:
        delete_rocket_list.reverse()
        delete_enemy_list.reverse()
        
        for dr in delete_rocket_list:
            del rocket_list[dr]
        for de in delete_enemy_list:
            del enemy_list[de]
    except:
        pass
    
    for i in range(len(enemy_list)):
        e = enemy_list[i]
        if crash(e, player) == True:
            system_exit = 1
            time.sleep(1)
        
    #  4 - 4) 전사작업(그리기)
    screen.fill(black)
    player.show()
    
    for r in rocket_list:
        r.show()
    
    for e in enemy_list:
        e.show()
    
    #  4 - 5) 업데이트
    pygame.display.flip()

# 5. 종료
pygame.quit()

1073741903
1073741903
1073741903
1073741903
1073741904
1073741904
1073741903
1073741903
1073741904
1073741904
1073741903
1073741903
1073741903
1073741903
